In [1]:
import numpy as np

nav_data = np.load(f'processed_nav_data.npy')
nav_labels = np.load(f'nav_data_labels.npy')
navs_columns = np.load('navs_columns_metadata.npy')

In [12]:
import pickle
from pprint import pprint
from sklearn import svm
from sklearn.model_selection import cross_validate

CV = 4

clf = svm.SVC(kernel = 'rbf', class_weight='balanced')
cv_results = cross_validate(clf, nav_data, nav_labels, cv=CV, scoring=('accuracy', 'recall_micro'))
print('SVM Radial Accuracy: ', cv_results['test_accuracy'].sum()/CV)
pprint(cv_results)

with open(f'nav_svm_rbf.pkl', 'wb') as f:
    pickle.dump(clf, f)

SVM Radial Accuracy:  0.7964285714285714
{'fit_time': array([0.04298425, 0.03017163, 0.03570628, 0.04265571]),
 'score_time': array([0.00938296, 0.00950336, 0.00971055, 0.01044512]),
 'test_accuracy': array([0.85      , 0.8       , 0.81071429, 0.725     ]),
 'test_recall_micro': array([0.85      , 0.8       , 0.81071429, 0.725     ])}
